In [1]:
import os, sys

In [34]:
archs

['3_16_mgga',
 '3_16_mgga_lr2',
 '3_16_nl',
 '3_16_nl_lr2',
 '3_16_nl_lr2_use2',
 '3_32_mgga',
 '3_32_mgga_lr2',
 '3_32_nl',
 '3_32_nl_lr2',
 '3_32_nl_lr2_use2',
 '4_16_mgga',
 '4_16_mgga_lr2',
 '4_16_nl',
 '4_16_nl_lr2',
 '4_16_nl_lr2_use2',
 '4_32_mgga',
 '4_32_mgga_lr2',
 '4_32_nl',
 '4_32_nl_lr2',
 '4_32_nl_lr2_use2']

In [38]:
TRAINDIR = '/home/awills/Documents/Research/xcquinox_pt/exc/pbe0'
VALDIR = '/home/awills/Documents/Research/xcquinox_pt/g297_val/exc/pbe0'
QUALS = ['nl', 'mgga']


tdirs = sorted([i for i in os.listdir(TRAINDIR) if any(j in i for j in QUALS) and os.path.isdir(os.path.join(TRAINDIR, i))])
archs = sorted(list(set([i[2:] for i in tdirs])))

CALC_SCRIPT = '~/xcquinox/scripts/calculate_traj.py'
G297_TRAJ = '/gpfs/scratch/awills/xcquinox_pt/g297_val/g2_97.traj'

XNP = ''
XNINP = ''
XNUSE = ''

CNP = ''
CNINP = ''
CNUSE = ''

GRID = 3

for ARCH in archs:
    xcs = sorted([i for i in tdirs if i[2:] == ARCH])
    print(xcs)
    for xca in xcs:
        runscrp = os.path.join(TRAINDIR, xca, 'run_script.sh')
        print(runscrp)
        with open(runscrp, 'r') as f:
            run = f.read()
        run_params = {i.split()[0].strip() : ' '.join(i.split()[1:]).strip() for i in run.split('--') if len(i.split()) > 1}
        print(run_params)
        if run_params['pretrain_net'] == 'c':
            CNP = os.path.join(TRAINDIR, xca)
            CNINP = run_params['n_input']
            CNUSE = run_params.get('use', None)
        elif run_params['pretrain_net'] == 'x':
            XNP = os.path.join(TRAINDIR, xca)
            XNINP = run_params['n_input']
            XNUSE = run_params.get('use', None)
    if CNUSE:
        run_script_text = f'''python {CALC_SCRIPT} {G297_TRAJ} \\
--atomize \\
--xc_x_net_path {XNP} \\
--xc_x_ninput {XNINP} \\
--xc_x_use {XNUSE} \\
--xc_c_net_path {CNP} \\
--xc_c_ninput {CNINP} \\
--xc_c_use {CNUSE} \\
--serial \\
--mf_grid_level {GRID}'''
    else:
        run_script_text = f'''python {CALC_SCRIPT} {G297_TRAJ} \\
--atomize \\
--xc_x_net_path {XNP} \\
--xc_x_ninput {XNINP} \\
--xc_x_use {XNUSE} \\
--xc_c_net_path {CNP} \\
--xc_c_ninput {CNINP} \\
--serial \\
--mf_grid_level {GRID}'''
        
    targvald = os.path.join(VALDIR, ARCH)
    print(targvald)
    try:
        os.mkdir(targvald)
    except Exception as e:
        print(e)
        print('DIRECTORY CREATION FAILED -- LIKELY EXISTS, OR MISSING INTERMEDIATE DIRECTORY')
    with open(os.path.join(targvald, 'run_script.sh'), 'w') as f:
        f.write(run_script_text)
    

['c_3_16_mgga', 'x_3_16_mgga']
/home/awills/Documents/Research/xcquinox_pt/exc/pbe0/c_3_16_mgga/run_script.sh
{'python': '~/xcquinox/scripts/pretrain_exc.py', 'pretrain_level': 'MGGA', 'pretrain_net': 'c', 'n_hidden': '16', 'depth': '3', 'n_input': '4', 'pretrain_xc': 'pbe0', 'n_steps': '2500', 'g297_data_path': '/gpfs/home/awills/xcquinox/scripts/script_data/haunschild_g2/g2_97.traj'}
/home/awills/Documents/Research/xcquinox_pt/exc/pbe0/x_3_16_mgga/run_script.sh
{'python': '~/xcquinox/scripts/pretrain_exc.py', 'pretrain_level': 'MGGA', 'pretrain_net': 'x', 'n_hidden': '16', 'depth': '3', 'n_input': '2', 'use': '1 2', 'pretrain_xc': 'pbe0', 'lr_decay_start': '500', 'n_steps': '2500', 'g297_data_path': '/gpfs/home/awills/xcquinox/scripts/script_data/haunschild_g2/g2_97.traj'}
/home/awills/Documents/Research/xcquinox_pt/g297_val/exc/pbe0/3_16_mgga
[Errno 17] File exists: '/home/awills/Documents/Research/xcquinox_pt/g297_val/exc/pbe0/3_16_mgga'
DIRECTORY CREATION FAILED -- LIKELY EXISTS, 

In [24]:
run_script_text

'python ~/xcquinox/scripts/calculate_traj.py /gpfs/scratch/awills/xcquinox_pt/g297_val/g2_97.traj --atomize --xc_x_net_path /home/awills/Documents/Research/xcquinox_pt/exc/pbe0/x_4_32_nl_lr2_use2 --xc_x_ninput 13 --xc_x_use 1 2 3 6 7 8 9 10 11 12 13 14 15 --xc_c_net_path /home/awills/Documents/Research/xcquinox_pt/exc/pbe0/c_4_32_nl_lr2_use2 --xc_c_ninput 14 --xc_c_use 0 1 2 3 6 7 8 9 10 11 12 13 14 15 --serial --mf_grid_level 3\n'